In [2]:
import pandas as pd
import numpy as np
from single_variable_optimizer import SingleVariableOptimizer

In [3]:
# build an example to test the optimizer
def test_single_variable_optimizer():
    
    # Define the objective function
    def objective_function(x):
        return x**2 - 4*x + 4

    # Define the range
    a = 0
    b = 4

    # Initialize the optimizer
    optimizer = SingleVariableOptimizer(objective_function, a, b, n_steps=20, e=0.01, minimize=True)

    # Run the optimizer
    result = optimizer.optimize()
    print(result)

test_single_variable_optimizer()

(1.9999984268177553, 2.474909166494399e-12)


In [24]:
class MultiVariableOptimizer:
    
    def __init__(self, objective_function, n_vars, a, b, termination_criteria=[0.01,0.0001,0.0001], minimize=True):
        self.objective_function = objective_function  # Objective function
        self.current_position = self.random_initializer(n_vars, a, b)  # Initial guess
        self.a = a  # Lower bound
        self.b = b  # Upper bound
        self.termination_criteria = termination_criteria  # [ε1, ε2, ε3]
        self.minimize = minimize  # Boolean: True if objective is to minimize
        self.function_call_counter = 0  # Iteration counter
        self.logger = []

    def call_objective_function(self, x):
        self.function_call_counter += 1
        return self.objective_function(x)
    
    def random_initializer(self, n, a, b):
        value = np.random.uniform(a, b)
        return np.full(n, value)
    
    def gradient(self, x):
        grad = np.zeros_like(x, dtype=float)
        h = 1e-5  # Step size for numerical differentiation
        for i in range(len(x)):
            x1 = np.copy(x)
            x2 = np.copy(x)
            x1[i] += h
            x2[i] -= h            
            grad[i] = (self.call_objective_function(x1) - self.call_objective_function(x2)) / (2*h)
        return grad
    
    def line_search(self, x, conjugate_direction, epsilon1):
        # Normalize the conjugate direction
        conjugate_direction = conjugate_direction / np.linalg.norm(conjugate_direction)
        
        def one_dimensional_objective_function(Lambda: float):
            Lambda = np.full_like(x, Lambda)
            return self.call_objective_function(x + Lambda * conjugate_direction)
        
        lowerX = np.full_like(x, self.a)
        upperX = np.full_like(x, self.b)
        alphaL = (lowerX-x)/conjugate_direction
        alphaU = (upperX-x)/conjugate_direction
        lower = np.max(alphaL)
        upper = np.min(alphaU)
        
        opt = SingleVariableOptimizer(one_dimensional_objective_function, lower, upper, 20 ,epsilon1)  # Initialize the optimizer
        optimal_lambda, _ = opt.optimize()
        return optimal_lambda

    def optimize(self):
        self.logger.append("--------------------------------------------------------------------------")
        # Step 1: Initialization
        self.logger.append("INITIATING CONJUGATE GRADIENT METHOD")
        k = 0 # max iterations = 100
        epsilon1, epsilon2, epsilon3 = self.termination_criteria
        x = self.current_position # initial position

        # Step 2: Compute the gradient at x
        self.logger.append(f"Initial guess: {x}")
        grad = self.gradient(x)
        s = -1 * np.array(grad) 
        self.logger.append(f"Initial gradient: {grad}")
        self.logger.append(f"Initial conjugate direction: {s}")

        # Step 3: Unidirectional search
        self.logger.append("UNDIRECTIONAL SEARCH")
        Lambda = self.line_search(x, s, epsilon1)
        self.logger.append(f"Initial Lambda: {Lambda}")
        
        x_next = x + Lambda * s/np.linalg.norm(s)
        self.logger.append(f"Next position: {x_next}")
        k += 1
               
        while True:

            grad_next = self.gradient(x_next)

            self.logger.append("--------------------------------------------------------------------------")
            self.logger.append(f"Iteration {k} : \n x = {x_next}, \n grad = {grad_next} , \n prev grad = {grad}, \n conjugate direction = {s}")
            # Step 4: Update the Conjugate direction
            s_next = (-1 * np.array(grad_next)) + (np.linalg.norm(grad_next) ** 2) / (np.linalg.norm(grad) ** 2) * np.array(s)

            self.logger.append(f"Conjugate direction: {s_next}")
            # Step 5: Unidirectional search
            Lambda = self.line_search(x_next, s_next, epsilon1)
            self.logger.append(f"Lambda: {Lambda}")
            
            x_next = x_next + Lambda * np.array(s_next)/np.linalg.norm(s_next)

            self.logger.append(f"Next position: {x_next}")

            # Check linear independence of the conjugate directions
            dot_product = np.dot(s, s_next)
            angle = np.arccos(dot_product / (np.linalg.norm(s) * np.linalg.norm(s_next)))
            if angle < np.deg2rad(5):
                self.logger.append("Conjugate directions are linearly dependent. Restarting the method")
                return self.optimize()  
            
            # Step 6: Check termination criteria
            if np.linalg.norm(x_next - x) / np.linalg.norm(x) <= epsilon2:
                self.logger.append("Termination condition met: Relative change in x is within epsilon2")
                return x_next  # Return optimal solution

            if np.linalg.norm(grad_next) <= epsilon3:
                self.logger.append("Termination condition met: Gradient norm is within epsilon3")
                return x_next  # Return optimal solution

            if k == 500:
                self.logger.append("Termination condition met: Maximum number of iterations reached")
                return x_next  # Return optimal solution

            self.logger.append("Move to the next iteration")
            # Move to the next iteration
            s = s_next
            grad = grad_next
            x = x_next
            k += 1
            
    def log(self):
        return '\n'.join(self.logger)

In [22]:
def sum_squares(x):
    return sum([(i+1)*x[i] ** 2 for i in range(len(x))])

def rosenbrock(x):
    return sum([100 * (x[i+1] - x[i]**2)**2 + (x[i] - 1)**2 for i in range(len(x) - 1)])

def dixon_price(x):
    n = len(x)
    return (x[0] - 1)**2 + sum([(i + 1) * (2 * x[i]**2 - x[i-1])**2 for i in range(1, n)])

def trid(x):
    n = len(x)
    return sum([(x[i] - 1)**2 for i in range(n)]) - sum([x[i] * x[i-1] for i in range(1, n)])

def zakharov_function(x):
    d = len(x)
    sum1 = np.sum(x**2)
    sum2 = np.sum(0.5 * (np.arange(1, d + 1) * x))
    
    return sum1 + sum2**2 + sum2**4

SUM OF SQUARES

In [89]:
# Sum of Squares Function
optimizer = MultiVariableOptimizer(sum_squares, 5, -10, 10)
optimal_solution = optimizer.optimize()

print(f"Optimal Solution: {np.round(optimal_solution, 2)}")
print(f"Optimal Value: {np.round(sum_squares(optimal_solution),2)}")
print(f"Function Calls: {optimizer.function_call_counter}")

Optimal Solution: [ 0.  0. -0.  0.  0.]
Optimal Value: 0.0
Function Calls: 373


In [90]:
print(optimizer.log())

--------------------------------------------------------------------------
INITIATING CONJUGATE GRADIENT METHOD
Initial guess: [-0.97068089 -0.97068089 -0.97068089 -0.97068089 -0.97068089]
Initial gradient: [-1.94136179 -3.88272358 -5.82408537 -7.76544716 -9.70680895]
Initial conjugate direction: [1.94136179 3.88272358 5.82408537 7.76544716 9.70680895]
UNDIRECTIONAL SEARCH
Initial Lambda: 1.7600923487943714
Next position: [-0.73335009 -0.49601929 -0.25868848 -0.02135768  0.21597312]
--------------------------------------------------------------------------
Iteration 1 : 
 x = [-0.73335009 -0.49601929 -0.25868848 -0.02135768  0.21597312], 
 grad = [-1.46670018 -1.98407715 -1.5521309  -0.17086143  2.15973125] , 
 prev grad = [-1.94136179 -3.88272358 -5.82408537 -7.76544716 -9.70680895], 
 conjugate direction = [1.94136179 3.88272358 5.82408537 7.76544716 9.70680895]
Conjugate direction: [ 1.59023591  2.2311486   1.92273807  0.66500433 -1.54205262]
Lambda: 0.6635780951560024
Next position

ROSENBROCK FUNCTION

In [23]:
# Rosenbrock Function
optimizer = MultiVariableOptimizer(rosenbrock, 3, -2.048, 2.048)
optimal_solution = optimizer.optimize()

print(f"Optimal Solution: {np.round(optimal_solution, 2)}")
print(f"Optimal Value: {np.round(rosenbrock(optimal_solution),2)}")
print(f"Function Calls: {optimizer.function_call_counter}")

Optimal Solution: [1. 1. 1.]
Optimal Value: 0.0
Function Calls: 12080


In [25]:
print(optimizer.log())

--------------------------------------------------------------------------
INITIATING CONJUGATE GRADIENT METHOD
Initial guess: [0.97, 0.97, 0.97]
Initial gradient: [-11.35079996  -5.53079996   5.82      ]
Initial conjugate direction: [11.35079996  5.53079996 -5.82      ]
UNDIRECTIONAL SEARCH
Initial Lambda: 0.024341718458452382
Next position: [0.98987278 0.97968323 0.95981045]
--------------------------------------------------------------------------
Iteration 1 : 
 x = [0.98987278 0.97968323 0.95981045], 
 grad = [ 0.0450351  -0.08584729  0.00624439] , 
 prev grad = [-11.35079996  -5.53079996   5.82      ], 
 conjugate direction = [11.35079996  5.53079996 -5.82      ]
Conjugate direction: [-0.04448096  0.0861173  -0.00652852]
Lambda: 7.498085712557691e-05
Next position: [0.98983845 0.9797497  0.95980541]
Move to the next iteration
--------------------------------------------------------------------------
Iteration 2 : 
 x = [0.98983845 0.9797497  0.95980541], 
 grad = [-0.00826375 -0.

DIXON PRICE FUNCTION

In [65]:
# Dixon Price Function
optimizer = MultiVariableOptimizer(dixon_price, 4, -10, 10)
optimal_solution = optimizer.optimize()
print(f"Optimal Solution: {np.round(optimal_solution, 2)}")
print(f"Optimal Value: {np.round(dixon_price(optimal_solution),2)}")
print(f"Function Calls: {optimizer.function_call_counter}")

Optimal Solution: [1.   0.71 0.59 0.55]
Optimal Value: 0.0
Function Calls: 1774


In [66]:
print(optimizer.log())

--------------------------------------------------------------------------
INITIATING CONJUGATE GRADIENT METHOD
Initial guess: [7.53144632 7.53144632 7.53144632 7.53144632]
Initial gradient: [ -410.59279101 12127.4766323  18297.12887047 25525.92031716]
Initial conjugate direction: [   410.59279101 -12127.4766323  -18297.12887047 -25525.92031716]
UNDIRECTIONAL SEARCH
Initial Lambda: 12.140858328756197
Next position: [ 7.67950383  3.15834466  0.93360186 -1.67306075]
--------------------------------------------------------------------------
Iteration 1 : 
 x = [ 7.67950383  3.15834466  0.93360186 -1.67306075], 
 grad = [ -35.72410515  628.57626854  -69.02510499 -249.73644901] , 
 prev grad = [ -410.59279101 12127.4766323  18297.12887047 25525.92031716], 
 conjugate direction = [   410.59279101 -12127.4766323  -18297.12887047 -25525.92031716]
Conjugate direction: [  35.89199195 -633.53505805   61.54361372  239.29918579]
Lambda: 1.383834770335402
Next position: [ 7.75244289  1.87088615  1.0

TRID Function

In [30]:
optimizer = MultiVariableOptimizer(trid, 6, -36, 36)
optimal_solution = optimizer.optimize()
print(f"Optimal Solution: {np.round(optimal_solution, 2)}")
print(f"Optimal Value: {np.round(trid(optimal_solution),2)}")
print(f"Function Calls: {optimizer.function_call_counter}")

Optimal Solution: [ 6. 10. 12. 12. 10.  6.]
Optimal Value: -50.0
Function Calls: 255


In [31]:
print(optimizer.log())

--------------------------------------------------------------------------
INITIATING CONJUGATE GRADIENT METHOD
Initial guess: [27.88477406 27.88477406 27.88477406 27.88477406 27.88477406 27.88477406]
Initial gradient: [25.8847741  -1.99999999 -1.99999999 -1.99999999 -1.99999999 25.8847741 ]
Initial conjugate direction: [-25.8847741    1.99999999   1.99999999   1.99999999   1.99999999
 -25.8847741 ]
UNDIRECTIONAL SEARCH
Initial Lambda: 17.247949486384663
Next position: [15.76079688 28.82153922 28.82153922 28.82153922 28.82153922 15.76079688]
--------------------------------------------------------------------------
Iteration 1 : 
 x = [15.76079688 28.82153922 28.82153922 28.82153922 28.82153922 15.76079688], 
 grad = [ 0.70005458 11.06074235 -1.99999999 -1.99999999 11.06074233  0.70005453] , 
 prev grad = [25.8847741  -1.99999999 -1.99999999 -1.99999999 -1.99999999 25.8847741 ], 
 conjugate direction = [-25.8847741    1.99999999   1.99999999   1.99999999   1.99999999
 -25.8847741 ]
Con

ZAKHAROV FUNCTION

In [34]:
optimizer = MultiVariableOptimizer(zakharov_function, 2, -5, 10)
optimal_solution = optimizer.optimize()
print(f"Optimal Solution: {np.round(optimal_solution, 2)}")
print(f"Optimal Value: {np.round(zakharov_function(optimal_solution),2)}")
print(f"Function Calls: {optimizer.function_call_counter}")

Optimal Solution: [0. 0.]
Optimal Value: 0.0
Function Calls: 375


In [35]:
print(optimizer.log())

--------------------------------------------------------------------------
INITIATING CONJUGATE GRADIENT METHOD
Initial guess: [-3.00181002 -3.00181002]
Initial gradient: [-193.0864097  -380.16919935]
Initial conjugate direction: [193.0864097  380.16919935]
UNDIRECTIONAL SEARCH
Initial Lambda: 4.031116615409359
Next position: [-1.17637233  0.59230718]
--------------------------------------------------------------------------
Iteration 1 : 
 x = [-1.17637233  0.59230718], 
 grad = [-2.3486235   1.19285666] , 
 prev grad = [-193.0864097  -380.16919935], 
 conjugate direction = [193.0864097  380.16919935]
Conjugate direction: [ 2.35599277 -1.17834725]
Lambda: 1.3170681881840351
Next position: [0.0015791  0.00315604]
Move to the next iteration
--------------------------------------------------------------------------
Iteration 2 : 
 x = [0.0015791  0.00315604], 
 grad = [0.0071039  0.01420351] , 
 prev grad = [-2.3486235   1.19285666], 
 conjugate direction = [ 2.35599277 -1.17834725]
Conj